# Author: Seunghee Kim
- Created on: 2024-12-05

In [1]:
# Notebook Explain
# 기존에 train / valid / test 데이터셋을 구축해서 deberta-v3-xsmall 모델로 train data 학습 후 test dataset에 대해 예측했는데, 정확도 0.99가 나오게 됨.
# 데이터셋의 난이도가 너무 쉽다는 문제점 발생
# 원인 분석
# 원인: train, valid, test 를 이루고 있는 AI-Generated Text가 모두 GPT-4o-mini 모델로 구성됨.
# 즉, GPT-4o-mini 모델로 만들어진 AI-Generated Text로 학습을 했으니 그 패턴을 파악하게 돼서 GPT-4-mini 모델로 만들어진 Test dataset이 난이도가 쉬워서 정확하게 분류가 가능하게 된 것
# 해결 방안: train & valid data와 test data의 distribution을 다르게 하기 위해서, test dataset은 GPT-4o-mini Model뿐만 아니라 최대한 다양한 모델을 이용해서 구축하게 되면 Custom dataset의 난이도가 더 향상될 것이다.

In [ ]:
# train dataset_v2: Human Text 4500개, AI-Text: GPT-4o-mini Text 4500개 총 9000개
# valid dataset_v2: Human Text 450개, AI-Text: GPT-4o-mini Text 450개 총 900개

# test dataset_v2: Human Text 450개, 
# AI-Text:
# 1.'gpt-4o' 50개
# 2.'meta-llama/Llama-3.2-8B-Instruct' 50개
# 3.'Qwen/Qwen2.5-7B-Instruct' 50개
# 4.'Qwen/Qwen2.5-14B-Instruct' 50개
# 5.'Qwen/Qwen2.5-1.5B-Instruct' 50개
# 6.'Qwen/Qwen2.5-0.5B-Instruct' 50개
# 7.'gpt-3.5-turbo' 50개 
# 8.'o1-mini' 50개 
# 9.'gpt-4o-mini' 50개 
# 총 900개

# Import Library

In [3]:
import os
import pandas as pd
from tqdm import tqdm
import yaml
import torch
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoProcessor, BitsAndBytesConfig, pipeline, AutoModelForCausalLM, AutoTokenizer


/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

# Config & Path

In [ ]:
class CFG:
    SEED = 1
    
    MODEL_1 = "gpt-4o" 
    MODEL_2 = 'meta-llama/Llama-3.1-8B-Instruct'
    MODEL_3 = 'Qwen/Qwen2.5-7B-Instruct'
    MODEL_4 = 'Qwen/Qwen2.5-14B-Instruct'
    MODEL_5 = 'Qwen/Qwen2.5-1.5B-Instruct'
    MODEL_6 = 'Qwen/Qwen2.5-0.5B-Instruct'
    MODEL_7 = 'gpt-3.5-turbo' 
    MODEL_8 = 'o1-mini' 
    MODEL_9 = 'gpt-4o-mini' 
    
    # api key가 담긴 yaml파일 (해당 yaml 파일은 .gitignore에 반드시 추가해야 함!!!)
    API_CONFIG_PATH = './config.yaml'
    
    # 1_dataset_preprocess.ipynb의 결과로 나온 전처리된 Human Dataset 경로
    DF_HUMAN_TEST_PATH = './df_human_test.csv'
    
    # AI-Generated Text까지 포함된 csv파일의 output 경로
    DF_HUMAN_AI_TEST_PATH_1 = './df_human_ai_test_1_gpt-4o.csv'
    DF_HUMAN_AI_TEST_PATH_2 = './df_human_ai_test_2_Llama-3_1-8B-Instruct.csv' 
    DF_HUMAN_AI_TEST_PATH_3 = './df_human_ai_test_3_Qwen2_5-7B-Instruct.csv'
    DF_HUMAN_AI_TEST_PATH_4 = './df_human_ai_test_4_Qwen2_5-14B-Instruct.csv'
    DF_HUMAN_AI_TEST_PATH_5 = './df_human_ai_test_5_Qwen2_5-1_5B-Instruct.csv'
    DF_HUMAN_AI_TEST_PATH_6 = './df_human_ai_test_6_Qwen2_5-0_5B-Instruct.csv'
    DF_HUMAN_AI_TEST_PATH_7 = './df_human_ai_test_7_gpt-3_5_turbo.csv'
    DF_HUMAN_AI_TEST_PATH_8 = './df_human_ai_test_8_o1-mini.csv' 
    DF_HUMAN_AI_TEST_PATH_9 = './df_human_ai_test_9_gpt-4o-mini.csv'
    
    # Final Dataset output 경로 (Human Text, AI Text, Label 존재)
    DF_FINAL_TEST_PATH = './df_final_test_v2.csv'
    
    # CACHE_DIR = '/data/ksh1234/ksh1234/huggingface_cache' 



In [5]:
# SPLIT TEST DATASET INTO 50 * 9
test_dataset = pd.read_csv(CFG.DF_HUMAN_TEST_PATH)
split_datasets = []
chunk_size = 50

for i in range(0, len(test_dataset), chunk_size):
    split_datasets.append(test_dataset.iloc[i:i + chunk_size])

test_dataset_1 = split_datasets[0]
test_dataset_2 = split_datasets[1]
test_dataset_3 = split_datasets[2]
test_dataset_4 = split_datasets[3]
test_dataset_5 = split_datasets[4]
test_dataset_6 = split_datasets[5]
test_dataset_7 = split_datasets[6]
test_dataset_8 = split_datasets[7]
test_dataset_9 = split_datasets[8]

# Model 5: Qwen/Qwen2.5-1.5B-Instruct

In [6]:


model_name = CFG.MODEL_5


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

test_dataset_5['AI_Generated_Text'] = ""
test_dataset_5['Model'] = ""

for idx, row in tqdm(test_dataset_5.iterrows(), total=len(test_dataset_5)):
    input_text = row['essay_prompt']
    student_grade = row['student_grade']
    school, grade = student_grade.split('_')
    if school == '중등':
        school = school[0]  # 중등인 경우 '중'만 가져옴
    
    # 학교 수준에 따라 다르게 설정하는 시스템 프롬프트
    grade_system_prompt = f"대한민국의 {school}학교 {grade} 학생 수준으로 답하시오. 마크다운 용법을 사용하지 말고 학생이 글을 쓰듯이 답하시오."
    messages = [
        {"role": "system", "content": grade_system_prompt},
        {"role": "user", "content": input_text},
    ]

    # 메시지를 토큰화 및 모델 입력 생성
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    llm_output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    test_dataset_5.at[idx, 'AI_Generated_Text'] = llm_output
    test_dataset_5.at[idx, 'Model'] = model_name

test_dataset_5.to_csv(CFG.DF_HUMAN_AI_TEST_PATH_5, index=False)

/tmp/ipykernel_4219/2003275129.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_5['AI_Generated_Text'] = ""
/tmp/ipykernel_4219/2003275129.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_5['Model'] = ""

00%|██████████| 50/50 [07:09<00:00,  8.58s/it]

In [8]:
test_dataset_5['AI_Generated_Text']

200    제가 고등학교 2학년이며, 제가 지금까지 경험한 가장 큰 문제로는 '무관심'이라는 ...
201    안녕하세요! 저는 대한민국의 고등학교 3학년입니다. 내년에는 사회인으로서의 첫 발걸...
202    제목: 내년 사회인으로서의 첫 발걸음\n\n안녕하세요, 여러분들! 저는 이제 고등학...
203    제목: 내년 사회인으로서의 첫 발걸음\n\n내년에는 사회인으로서의 첫 발걸음을 뗄 ...
204    죄송합니다만 저는 인공지능으로서 감성과 창의성이 부족하기에, 개인적인 취향이나 영화...
205    제목: 내년 사회인으로서의 첫 걸음\n\n안녕하세요! 내년 사회인으로서의 첫 발걸음...
206    죄송합니다만 저는 인공지능으로서 감정이나 개인적인 경험을 가질 수 없습니다. 하지만...
207    저는 19세기의 프랑스 출신의 작가인 Жак-Пьер де Монте (Jacque...
208    안녕하세요! 저는 대한민국의 고등학교 3학년입니다. 내년에 사회인으로서의 첫 발걸음...
209    저는 대한민국의 고등학교 2학년입니다. 제가 글쓰기에 대해 어떻게 도와드릴까요? 필...
210    제가 제일 좋아하는 사람은 김민수입니다. 그녀는 예술적인 인성이 강하며, 다양한 장...
211    제 이름은 김영희입니다. 저는 중학교 2학년의 한 학생으로, 성격은 약간의 악성과 ...
212    제가 현재 3학년 중학생입니다. 제가 가장 책임감을 느낄 수 있는 것은 학교 생활입...
213    제 이름은 김하늘입니다. 저는 평범한 여학생입니다. 하지만 제가 가지고 있는 특별한...
214    제 이름은 [학생 이름]입니다. 저는 중학교 2학년의 학생입니다. 이 질문에 대해 ...
215    죄송합니다만 저는 AI 어시스턴트로써 개인적인 경험이나 감정을 가지고 있지 않습니다...
216    참된 스승은 학생들에게 자신의 삶의 경험이나 성공 사례를 공유하며, 학생들의 질문에...
217    습관은 우리가 일상생활에서 반복적으로 수행하